# Data science Project

## Intro

In this project we going to explor data about ELAL's flight history.
The perpose is to explor if we can to predict if ELAL's flight will delayd or canceld



### Imports

In [643]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import time
from dateutil.relativedelta import relativedelta, MO
from dateutil.easter import easter
import jewish
# Credit to https://github.com/emosenkis/jewish/blob/master/LICENSE

### Const Varibles

In [644]:
BASE_URL='https://api.flightstats.com/flex/flightstatus/historical/rest/v3/json/'


# APP_KEY='1073eea843b49ef65cd3dedce57340e0'
# APP_ID='a55ccedd'

# APP_KEY='9981b617cba5bb3f32d90ba0da4e2af4'
# APP_ID='65c679ae'

# APP_KEY= '9fb822148287f2805212cc61c199d8f8'
# APP_ID = 'd7585437'

# APP_KEY = 'eca188e5f3d730ee5d28907112889342'
# APP_ID = '4176580c'

APP_KEY = '37512902d28a8deac5ec0325e9f40a56'
APP_ID = 'bfeb4319'

ElAl_AIRCRAFTS_WEB_URL = 'https://www.elal.com/heb/about-elal/our-fleet/all-aircraft'
FLIGHTEAR_Base_URL = 'https://www.flightera.net/en/planes/'

AIRCRAFTS_DATA_FILE = 'aircrafts.json'
FLIGHT_Data_FILE = 'flights_data.json'
FLIGHTS_SCAN_METADATA_FILE = 'flights_scan_metadata.json'

TAIL_NUMBER_FEILD_NAME = 'tail_number'
FIRST_FLIGHT_DATE_FEILD_NAME = 'first_flight_date'
LAST_FLIGHT_DATE_FEILD_NAME = 'last-flight_date'
SEATS_FEILD_NAME = 'seats'
LAST_SCANNED__DATE_FEILD_NAME = 'last_scanned_date'


## General files and json functions

Functions that store and loads data from files

In [645]:
def store_object_to_file(file_name,data):
    with open(file_name, 'w') as outfile:
        json.dump(data, outfile)

In [646]:
def load_object_from_file(filename):
    if(not os.path.exists(filename)):
        return None
    with open(filename) as f:
        return json.load(f)

In [647]:
def store_data_to_aircrafts_file(tailnumber,feild_name,data):
    aircrafts_list = load_object_from_file(AIRCRAFTS_DATA_FILE)
    if(not aircrafts_list):
        aircrafts_list = []
    for i,aircraft in enumerate(aircrafts_list):
        if(tailnumber == aircraft[TAIL_NUMBER_FEILD_NAME]):
            aircrafts_list[i][feild_name] = data
            store_object_to_file(AIRCRAFTS_DATA_FILE,aircrafts_list)
            return

In [648]:
def load_data_from_aircrafts_file(tailnumber,feild_name):
    if(not os.path.exists(AIRCRAFTS_DATA_FILE)):
        return None
    aircrafts_list = load_object_from_file(AIRCRAFTS_DATA_FILE)
    for aircraft in aircrafts_list:
        if(tailnumber == aircraft[TAIL_NUMBER_FEILD_NAME]):
            if feild_name in aircraft:
                return aircraft[feild_name]
            else:
                return None

## Dates and holidays calculations

Mounths dictianery used to translate month name to numbrr

In [649]:
months = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }


Check if some date is crestian holiday

In [650]:
def is_christian_holiday(date):
    easter_date = easter(date.year)
    easter_sunday = datetime.strptime(easter_date.strftime('%m/%d/%Y'), '%m/%d/%Y').date()
    easter_monday = easter_sunday + relativedelta(weekday=MO)
    good_friday = easter_sunday - relativedelta(days=2)
    christmas_day = datetime(year=date.year, month=12, day=25).date()
    holidays = [easter_sunday, easter_monday, good_friday, christmas_day]
    return int(date in holidays)

In [651]:
def is_jewish_holiday(date):
    jewish_date = jewish.JewishDate.from_date(date)

    #rosh hashana
    if(jewish_date.day == 1 and jewish_date.month == 1): return 1
    #yom kipur
    if(jewish_date.day == 10 and jewish_date.month == 1): return 1
    #sukot
    if(jewish_date.month == 1 and (jewish_date.day >= 16 and jewish_date.day <= 21)): return 1
    #hanuka
    if(jewish_date.month == 3 and (jewish_date.day >= 23 and jewish_date.day <= 29)): return 1
    #passover
    if(jewish_date.month == 7 and (jewish_date.day >= 14 and jewish_date.day <= 22)): return 1

    return 0



### API functions

In [652]:
# "BASE URL + flight/status/tailNumber/4x-ELA/2015/1/1?appId=b1aa7a60&appKey=ed14bb251c167a18733cccbf86105175"

def get_Flight_by_date_and_tail_number(date,tail_number, days_per_request):
    formatted_date = date.strftime("%Y/%m/%d")

    #URL = BASE_URL + 'flight/status/tailNumber/' + tail_number + '/' + formatted_date + '?appId=' + APP_KEY + '&appKey=' + API_KEY
    URL = BASE_URL+'flight/status/tailNumber/'+tail_number+'/'+formatted_date+'?appId='+APP_ID+'&appKey='+APP_KEY+'&numDays=30'#+str(days_per_request)
    response = requests.get(URL)
    json_res =  response.json()
    if 'flightStatuses' in json_res: return json_res['flightStatuses'] , None
    if('error' in json_res): return None , json_res['error']
    return None, json_res[0]


test the api

In [653]:
# res = get_Flight_by_date_and_tail_number(datetime(2017,9,28),'4X-EDA',days_per_request=30)
# res

## Get information that we need before start to send api requests

To send this API request we have to know the aircradt tail number
we will get this informaition from elal web side via crouling

In [654]:
elal_web_res = requests.get(ElAl_AIRCRAFTS_WEB_URL)

In [655]:

elal_soup = BeautifulSoup(elal_web_res.content)

Define functios that clean the data


The format of right israeli tail number is '4X-' and 3 letters
like: '$X-ABC'

In [656]:
def clean_tail_numberst(tail_number):
    tail_number = tail_number.upper()
    first_instance_of_4x = tail_number.find('4X-')
    new_tail_number = tail_number[first_instance_of_4x:first_instance_of_4x+6]
    return new_tail_number



In [657]:
def clean_aircraft_name(name):
    return str(name.replace('\u200b',''))

In [658]:
def clean_model(model):
    model = model.strip()
    return model[3:]

Define function that return list of aircraft with ther usefull data

In [659]:
def get_basic_aircrafts_data_list():
    if(os.path.exists(AIRCRAFTS_DATA_FILE)):
        return load_object_from_file(AIRCRAFTS_DATA_FILE)
    
    aircrafts_list=[]
    tables_list = elal_soup.find_all('tbody')
    headers_list = elal_soup.find_all('span',class_ = 'mat-content')
    for j,table in enumerate(tables_list):
        rows_list = table.findAll('tr')
        for row in rows_list[1:]:
            cells_list = row.findAll('td')
            for i,cell in enumerate(cells_list[1::2]):
                aircraft_object = {}
                aircraft_object['tail_number'] = clean_tail_numberst(cell.get_text())
                aircraft_object['model'] = clean_model(headers_list[j].get_text())
                aircraft_object['name'] = clean_aircraft_name(cells_list[i].get_text())
                aircrafts_list.append(aircraft_object) 

    if (not os.path.exists(AIRCRAFTS_DATA_FILE)):
        store_object_to_file(AIRCRAFTS_DATA_FILE,aircrafts_list)   
    return aircrafts_list

#basic_data



### Get more Advanced information about aircraft that not exost in ELAL website 

this dictianery store soap object about each aircraft
it use to lower the http get request to flightera website becaue it will block us if we try to request each time we want more information about the same aircraft

In [660]:
flightEra_soaps_dict = {}

Define function that get flightera soap object of spesific air craft

if we also have soap object about this aircraft in the dict we just need to return it
else, we create http request, convert it to soap object and store it to the dist. then return it

In [661]:
def get_flghtera_soap_by_tail_number(tail_number): 
    if (tail_number.lower() not in flightEra_soaps_dict):       
        URL = FLIGHTEAR_Base_URL + tail_number
        flightera_response = requests.get(URL)
        if (not flightera_response.status_code == 200):
            print('Error resived data about '+tail_number+' from FLightera web sit, error code: ' + str(flightera_response.status_code))
            return None
        flightEra_soaps_dict[tail_number] =  BeautifulSoup(flightera_response.content)
    
    return flightEra_soaps_dict[tail_number]

In [662]:
x = get_flghtera_soap_by_tail_number('4x-elk')
x

Error resived data about 4x-elk from FLightera web sit, error code: 403


## Get first  and last filght date

we neet to get the first time that specific aircrat fly.
this information help us to reduse the api request we need to do because eithout it we will need to create requests for evwry day in pre known dates range and if we know the first and last flights date of the aircraft we can short the dates range

### Get first flight date

In [663]:
def get_first_flight_date_from_Cache(tail_number):
    return load_data_from_aircrafts_file(tail_number,FIRST_FLIGHT_DATE_FEILD_NAME)

In [664]:
def store_first_flight_date_data(tail_number,first_flight_date):
    store_data_to_aircrafts_file(tail_number,FIRST_FLIGHT_DATE_FEILD_NAME,first_flight_date)

In [665]:
def clean_first_flight_dtate_string(date_str):
    clean_str = date_str.replace('\n', '')
    clean_str = clean_str.strip()
    clean_str = clean_str[:clean_str.find('(')]
    clean_str = clean_str.strip()
    return clean_str

In [666]:
def convert_daytime_to_dict(date):
    if (date == None):
         return None
    return {
        'year' : date.year,
        'month' : date.month,
        'day' : date.day
    }

In [667]:
def get_first_flight_date_by_crowling(tail_number):
    aircraft_soap = get_flghtera_soap_by_tail_number(tail_number.lower())
    if(not aircraft_soap): 
        return None
    try:
        for td in aircraft_soap.find_all('td'):
            if(td.get_text().find('years ago') != -1):
                date_string = td.get_text()
        date_string = clean_first_flight_dtate_string(date_string)
        month , year = date_string.split(' ')
        return convert_daytime_to_dict(datetime(int(year),int(months[month.lower()]),27))
    except:
        return None
    return None

In [668]:
def get_first_flight_date_by_tailnnumber(tail_number):
    cashed_data = get_first_flight_date_from_Cache(tail_number)
    
    if(cashed_data):
        return cashed_data
    else: 
        first_flight_date_data = get_first_flight_date_by_crowling(tail_number)
        if(first_flight_date_data):
            store_first_flight_date_data(tail_number,first_flight_date_data)
            return first_flight_date_data
        return None

### Get Last flight date

In [669]:
def get_last_flight_date_from_Cache(tail_number):
    return load_data_from_aircrafts_file(tail_number,LAST_FLIGHT_DATE_FEILD_NAME)

In [670]:
def store_last_flight_date_data(tail_number,last_flight_date):
    store_data_to_aircrafts_file(tail_number,LAST_FLIGHT_DATE_FEILD_NAME,last_flight_date)

In [671]:
def clean_last_flight_month(mnt):
    mnt = mnt.replace('\t','')
    mnt = mnt.strip()
    mnt = mnt[:3]
    return mnt.lower()

In [672]:
def get_last_flight_date_by_crowling(tail_number):
    aircraft_soap = get_flghtera_soap_by_tail_number(tail_number.lower())
    if(not aircraft_soap): return None
    try:
        year = int(aircraft_soap.find('body').find_all('div',class_='sm:px-6')[4].find_all('dt')[0].get_text())
        month = aircraft_soap.find('body').find_all('div',class_='sm:px-6')[4].find_all('dl')[0].find_all('dd')[0].find_all('div')[0].get_text()
        month = clean_last_flight_month(month)
        
        last_flight_date = datetime(year,months[month],1)
        return convert_daytime_to_dict(last_flight_date)
    except:
        return None
    return None

In [673]:
def get_last_flight_date_by_tailnnumber(tail_number):
    cashed_data = get_last_flight_date_from_Cache(tail_number)
    if(cashed_data): return cashed_data
    else: 
        last_flight_date_data = get_last_flight_date_by_crowling(tail_number)
        if(last_flight_date_data):
            store_last_flight_date_data(tail_number,last_flight_date_data)
            return last_flight_date_data
        return None

## Get number of seats per aircraft

this information can be usefull to predict the dilay time of flight

In [674]:
def store_seats_data(tail_number,seats_object):
    store_data_to_aircrafts_file(tail_number,SEATS_FEILD_NAME,seats_object)

In [675]:
def get_seats_from_Cache(tail_number):
    return load_data_from_aircrafts_file(tail_number,SEATS_FEILD_NAME)

In [676]:
def get_seats_by_crowling(tail_number):
    aircraft_soap = get_flghtera_soap_by_tail_number(tail_number.lower())
    aircraft_seat_object = {}
    strings = []
    try:
        for td in aircraft_soap.find_all('td'):
            if(td.get_text().find('seats') != -1):
                strings = td.get_text().split('\n')
                
    except:
        return None
    i=0
    for i,s in enumerate(strings):
        if(s.find('seats') != -1):
            break
    for content in strings[i+1:-1]:
        seat , class_ = content.split(' ')
        aircraft_seat_object[class_] = str(seat)
        print(aircraft_seat_object)

    return aircraft_seat_object
    

In [677]:
def get_seats_by_tail_number(tail_number):
    cashed_data = get_seats_from_Cache(tail_number)
    if(cashed_data): return cashed_data
    else: 
        seats_data = get_seats_by_crowling(tail_number)
        if(seats_data):
            store_seats_data(tail_number,seats_data)
            return seats_data
        return None
        

### Last scanned date

we store the last scanned date,
this date is the last date from all the days this air craft was active that we know the flight history of.

for example if the aircraft is active from 1/1/2017 untill today and last scanned date is 1/3/2017 it's mean we have all the flights of this aircraft from 1/1/2017 to 1/3/2017.

this information help us to avoid duplicate flights in our dataset and minimize the number of api requests

In [678]:
def get_last_scanned_date_from_Cache(tail_number):
    return load_data_from_aircrafts_file(tail_number,LAST_SCANNED__DATE_FEILD_NAME)
    

In [679]:
def get_last_scanned_date_of_aircraft(tail_number):
    cashed_data = get_last_scanned_date_from_Cache(tail_number)
    if(cashed_data): return cashed_data
    else: 
        last_scanned_date = load_data_from_aircrafts_file(tail_number,FIRST_FLIGHT_DATE_FEILD_NAME) 
        if(last_scanned_date):
            store_data_to_aircrafts_file(tail_number,LAST_SCANNED__DATE_FEILD_NAME,last_scanned_date)
            return last_scanned_date
        return None

## Put all this data togeter

Define function that get advanced information about eace air crafr
this function call to every function that can add information about spesific aircraft

In [680]:
def get_aircrafts_full_information():
    if (os.path.exists(AIRCRAFTS_DATA_FILE)):
        return load_object_from_file(AIRCRAFTS_DATA_FILE)
    
    aircrafts_list = get_basic_aircrafts_data_list()
    for aircraft in aircrafts_list:
        #Add first flight date information
        first_flght_date_data = get_first_flight_date_by_tailnnumber(aircraft[TAIL_NUMBER_FEILD_NAME])
        if(first_flght_date_data):
            aircraft[FIRST_FLIGHT_DATE_FEILD_NAME] = first_flght_date_data

        #Add last flight date information
        last_flght_date_data = get_last_flight_date_by_tailnnumber(aircraft[TAIL_NUMBER_FEILD_NAME])
        if(last_flght_date_data):
            aircraft[LAST_FLIGHT_DATE_FEILD_NAME] = last_flght_date_data

        #Add seats to airplanes data
        seats_data = get_seats_by_tail_number(aircraft[TAIL_NUMBER_FEILD_NAME])
        if(seats_data):
            aircraft[SEATS_FEILD_NAME] = seats_data
        
        last_scanned_daate = get_last_scanned_date_of_aircraft(aircraft[TAIL_NUMBER_FEILD_NAME])
        if(last_scanned_daate):
            aircraft[LAST_SCANNED__DATE_FEILD_NAME] = last_scanned_daate


    store_object_to_file(AIRCRAFTS_DATA_FILE,aircrafts_list)
    return aircrafts_list


We will add one more feild to the aircraft data that contain the last data that the API Checked

## Preview our aircrafts data

In [681]:
def get_airrcrafts_data_as_dataframe():
    aircrafts_list = get_aircrafts_full_information()
    df = pd.DataFrame(aircrafts_list)
    return df
aircrafts_df = get_airrcrafts_data_as_dataframe()
aircrafts_df

,tail_number,model,name,first_flight_date,last-flight_date,seats,last_scanned_date
0,4X-EKH,737-800,חדרה,"{'year': 2009, 'month': 4, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Business': '16', 'Economy': '138'}","{'year': 2016, 'month': 6, 'day': 24}"
1,4X-EKJ,737-800,דגניה,"{'year': 2009, 'month': 5, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Business': '16', 'Economy': '138'}","{'year': 2016, 'month': 9, 'day': 26}"
2,4X-EKL,737-800,נהלל,"{'year': 2009, 'month': 6, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Business': '16', 'Economy': '138'}","{'year': 2016, 'month': 10, 'day': 27}"
3,4X-EKT,737-800,בית שמש,"{'year': 2006, 'month': 6, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Eco+': '36', 'Economy': '144'}","{'year': 2016, 'month': 10, 'day': 27}"
4,4X-EKI,737-800,מודיעין מכבים רעות,"{'year': 1999, 'month': 1, 'day': 27}","{'year': 2023, 'month': 4, 'day': 1}","{'Business': '16', 'Economy': '150'}","{'year': 2017, 'month': 5, 'day': 26}"
5,4X-EKO,737-800,לוד,"{'year': 2003, 'month': 4, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}",{'Economy': '180'},"{'year': 2017, 'month': 8, 'day': 24}"
6,4X-EKP,737-800,נהריה,"{'year': 2001, 'month': 8, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Business': '16', 'Economy': '138'}","{'year': 2016, 'month': 12, 'day': 27}"
7,4X-EKS,737-800,קיסריה,"{'year': 2008, 'month': 8, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Business': '16', 'Economy': '138'}","{'year': 2016, 'month': 4, 'day': 1}"
8,4X-EKM,737-800,רמלה,"{'year': 2000, 'month': 2, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}","{'Eco+': '36', 'Economy': '144'}","{'year': 2015, 'month': 6, 'day': 4}"
9,4X-EKR,737-800,גבעתיים,"{'year': 2000, 'month': 2, 'day': 27}","{'year': 2023, 'month': 5, 'day': 1}",{'Economy': '185'},"{'year': 2015, 'month': 6, 'day': 4}"


## Start to collect data about flight 

Define function that collect data about aircraft between 2 dates

we use date as dict in the formate:
{
    'year' : YYYY,
    'month' : MM,
    'day' : DD
}

In [682]:
def get_dates_between(start_date, end_date):
    delta = end_date - start_date      
    date_list = []                     
    
    for i in range(delta.days + 1):     
        date_list.append(start_date + timedelta(days=i))   
    
    return date_list

In [683]:
def get_aircraft_flights_from_api(tail_number,max_number_of_days,days_per_request,limit_date = None,):
    aircrafts_list = get_aircrafts_full_information()
    flights_list = []
    if(os.path.exists(FLIGHT_Data_FILE)):
        flights_list = load_object_from_file(FLIGHT_Data_FILE)
    print('Collectiong '+tail_number+' flights')
    for aircraft in aircrafts_list:
        if(aircraft[TAIL_NUMBER_FEILD_NAME] == tail_number):
            first_flight_date_dict = None
            last_flight_date_dict = None
            first_flight_date = datetime(2000,1,27)
            last_flight_date = datetime.today()
            
            if(LAST_SCANNED__DATE_FEILD_NAME in aircraft):
                first_flight_date_dict = aircraft[LAST_SCANNED__DATE_FEILD_NAME]

            if(limit_date):
                last_flight_date = limit_date

            elif(LAST_FLIGHT_DATE_FEILD_NAME in aircraft):
                last_flight_date_dict = aircraft[LAST_FLIGHT_DATE_FEILD_NAME]

            if(first_flight_date_dict):
                first_flight_date = datetime(int(first_flight_date_dict['year']), int(first_flight_date_dict['month']),int(first_flight_date_dict['day']) )
            
            if(last_flight_date_dict):
                last_flight_date = datetime(int(last_flight_date_dict['year']),int(last_flight_date_dict['month']),int(last_flight_date_dict['day']) )
            for current_date in (get_dates_between(first_flight_date,last_flight_date)[:max_number_of_days:days_per_request]):
                #print(tail_number+' '+str(current_date))
                new_flights , error = get_Flight_by_date_and_tail_number(current_date,tail_number.upper(),days_per_request)
                if(new_flights != None):
                    flights_list = load_object_from_file(FLIGHT_Data_FILE)
                    flights_list.extend(new_flights)
                    store_data_to_aircrafts_file(aircraft[TAIL_NUMBER_FEILD_NAME],LAST_SCANNED__DATE_FEILD_NAME,convert_daytime_to_dict(current_date + timedelta(1)))
                    store_object_to_file(FLIGHT_Data_FILE,flights_list)
                else:
                    store_object_to_file(FLIGHT_Data_FILE,flights_list)
                    return flights_list , error
                    
            return flights_list , error



In [684]:
def get_all_aricrafts_flights(max_number_of_days,days_per_request):
    aircrafts_list = get_aircrafts_full_information()
    for aircraft in aircrafts_list:
        flights , error =get_aircraft_flights_from_api(aircraft[TAIL_NUMBER_FEILD_NAME],max_number_of_days,days_per_request)
        if(error):
            print(error)
            break
    return load_object_from_file(FLIGHT_Data_FILE)

In [685]:
# for i in range(1):
#     flights_list = get_all_aricrafts_flights(max_number_of_days=30,days_per_request=30)
#     len(flights_list)

In [686]:
# t = load_object_from_file(AIRCRAFTS_DATA_FILE)
# for ac in t:
#     ac[LAST_SCANNED__DATE_FEILD_NAME] = ac[FIRST_FLIGHT_DATE_FEILD_NAME]
# store_object_to_file(AIRCRAFTS_DATA_FILE,t)

# Data Exporation

## Intert all flights data into dataframe

In [687]:
def get_flights_dataframe():
    flights = load_object_from_file(FLIGHT_Data_FILE)
    return pd.DataFrame(flights)


## Preview the fights data

In [688]:
flights_df = get_flights_dataframe()

In [689]:

flights_df.head()

,flightId,carrierFsCode,operatingCarrierFsCode,primaryCarrierFsCode,flightNumber,departureAirportFsCode,arrivalAirportFsCode,departureDate,arrivalDate,status,schedule,operationalTimes,codeshares,delays,flightDurations,airportResources,flightEquipment,irregularOperations,divertedAirportFsCode
0,930925122,LY,LY,LY,1789,PAE,TLV,"{'dateUtc': '2017-08-22T18:15:00.000Z', 'dateL...","{'dateUtc': '2017-08-23T06:55:00.000Z', 'dateL...",L,"{'flightType': 'P', 'serviceClasses': 'RJY', '...",{'publishedDeparture': {'dateUtc': '2017-08-22...,[],{},"{'scheduledBlockMinutes': 760, 'airMinutes': 748}",{'arrivalTerminal': '3'},"{'scheduledEquipmentIataCode': '789', 'actualE...",[],NaN
1,932368916,LY,LY,LY,215,TLV,LHR,"{'dateUtc': '2017-09-04T08:35:00.000Z', 'dateL...","{'dateUtc': '2017-09-04T13:55:00.000Z', 'dateL...",L,"{'flightType': 'P', 'serviceClasses': 'RJY', '...",{'publishedDeparture': {'dateUtc': '2017-09-04...,[],{},"{'scheduledBlockMinutes': 320, 'airMinutes': 289}","{'departureTerminal': '3', 'arrivalTerminal': ...","{'scheduledEquipmentIataCode': '789', 'tailNum...",[],NaN
2,932368962,LY,LY,LY,216,LHR,TLV,"{'dateUtc': '2017-09-04T16:00:00.000Z', 'dateL...","{'dateUtc': '2017-09-04T20:50:00.000Z', 'dateL...",L,"{'flightType': 'P', 'serviceClasses': 'RJY', '...",{'publishedDeparture': {'dateUtc': '2017-09-04...,[],{},"{'scheduledBlockMinutes': 290, 'airMinutes': 248}","{'departureTerminal': '4', 'arrivalTerminal': ...","{'scheduledEquipmentIataCode': '789', 'tailNum...",[],NaN
3,932586208,LY,LY,LY,225,TLV,CDG,"{'dateUtc': '2017-09-06T16:15:00.000Z', 'dateL...","{'dateUtc': '2017-09-06T21:10:00.000Z', 'dateL...",L,"{'flightType': 'P', 'serviceClasses': 'RJY', '...",{'publishedDeparture': {'dateUtc': '2017-09-06...,[],{},{'scheduledBlockMinutes': 295},"{'departureTerminal': '3', 'arrivalTerminal': ...","{'scheduledEquipmentIataCode': '789', 'tailNum...",[],NaN
4,932696349,LY,LY,LY,226,CDG,TLV,"{'dateUtc': '2017-09-06T23:10:00.000Z', 'dateL...","{'dateUtc': '2017-09-07T03:35:00.000Z', 'dateL...",L,"{'flightType': 'P', 'serviceClasses': 'RJY', '...",{'publishedDeparture': {'dateUtc': '2017-09-06...,[],{},{'scheduledBlockMinutes': 265},"{'departureTerminal': '2A', 'arrivalTerminal':...","{'scheduledEquipmentIataCode': '789', 'tailNum...",[],NaN


In [690]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15817 entries, 0 to 15816
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   flightId                15817 non-null  int64 
 1   carrierFsCode           15817 non-null  object
 2   operatingCarrierFsCode  15817 non-null  object
 3   primaryCarrierFsCode    15817 non-null  object
 4   flightNumber            15817 non-null  object
 5   departureAirportFsCode  15817 non-null  object
 6   arrivalAirportFsCode    15817 non-null  object
 7   departureDate           15817 non-null  object
 8   arrivalDate             15817 non-null  object
 9   status                  15817 non-null  object
 10  schedule                15591 non-null  object
 11  operationalTimes        15817 non-null  object
 12  codeshares              15817 non-null  object
 13  delays                  15817 non-null  object
 14  flightDurations         15817 non-null  object
 15  ai

In [691]:
flights_df.describe()

,flightId
count,1.581700e+04
mean,9.173281e+08
std,1.684685e+08
min,6.059907e+08
25%,7.366493e+08
50%,9.658411e+08
75%,1.085159e+09
max,1.155650e+09


## Add aircraft data into flights data

We can add more information ablur each flight if we merge out air craft data with the aircraft used in the spesific flight

the next function add the air craft seats number dictionary as flight feature

In [692]:
def add_aircraft_seats_number_to_flight(flights_df, aircrafts_df):
    flights_df[SEATS_FEILD_NAME] = aircrafts_df[SEATS_FEILD_NAME]
    return flights_df

In [693]:
# flights_df =  add_aircraft_seats_number_to_flight(flights_df,aircrafts_df)
# flights_df

In [694]:
def get_arrivalGateDelayMinutes(delays_dict,feild):
    if feild in delays_dict:
        return delays_dict[feild]
    else:
        return 0

## Clean the resived tata from th api

Calculate the delay time  as sum of gate delay and runway delay
if after this calc there is na value we need drop it

In [695]:
def proccess_delay_column(df):
    if not 'delays' in df: return df
    df['departureRunwayDelayMinutes'] = df['delays'].apply(lambda dict: get_arrivalGateDelayMinutes(dict,'departureRunwayDelayMinutes'))
    df['departureGateDelayMinutes'] = df['delays'].apply(lambda dict: get_arrivalGateDelayMinutes(dict,'departureGateDelayMinutes'))
    df['arrivalGateDelayMinutes'] = df['delays'].apply(lambda dict: get_arrivalGateDelayMinutes(dict,'arrivalGateDelayMinutes'))
    df['arrivalRunwayDelayMinutes'] = df['delays'].apply(lambda dict: get_arrivalGateDelayMinutes(dict,'arrivalRunwayDelayMinutes'))
    df['delay_time'] = df['departureRunwayDelayMinutes'] + df['departureGateDelayMinutes'] + df['arrivalGateDelayMinutes'] +  df['arrivalRunwayDelayMinutes']
    #df['delay_time']=df['delay_time'].fillna(0)

    df = df.drop('departureRunwayDelayMinutes',axis = 1)
    df = df.drop('departureGateDelayMinutes',axis = 1)
    df = df.drop('arrivalGateDelayMinutes',axis = 1)
    df = df.drop('arrivalRunwayDelayMinutes',axis = 1)
    df= df.drop('delays',axis = 1)
    return df

In [696]:
def get_flightModel(dict):
    if 'actualEquipmentIataCode' in dict:
        return dict['actualEquipmentIataCode']
    elif 'scheduledEquipmentIataCode' in dict:
        return dict['scheduledEquipmentIataCode']
    return np.nan

In [697]:
def get_tailnumber(dict):
    if 'tailNumber' in dict:
        return dict['tailNumber']
    return np.nan

In [698]:
def get_flight_time(dict):
    if 'airMinutes' in dict:
        return dict['airMinutes']
    return np.nan

In [699]:
def get_departure_terminal(dict):
    if 'departureTerminal' in dict:
        return dict['departureTerminal']
    return np.nan

In [700]:
def get_arival_terminal(dict):
    if 'arrivalTerminal' in dict:
        return dict['arrivalTerminal']
    return np.nan

In [701]:
def get_utc_from_dict(df , col):
    if col not in df: return df
    df[col] = df[col].apply(lambda x: x['dateUtc'])
    df[col] = pd.to_datetime(df[col])
    return df

In [702]:
def proccess_date(df,col):
    if col not in df: return df
    df[col + '_year'] = df[col].dt.year
    df[col + '_month'] = df[col].dt.month
    df[col + '_day'] = df[col].dt.day
    df = df.drop(col,axis=1)
    return df

In [703]:
def extract_seat(json_str):
    try:
        data = json.loads(json_str)
        return pd.Series(data)
    except:
        return np.nan

In [704]:
def extract_seats(dict,class_):
    if class_ not in dict: return 0
    return dict[class_]

In [705]:
def get_dataframe_after_prossess(df):
    df = proccess_delay_column(df)
    df = df.drop('status',axis = 1)
    df = df.drop('operatingCarrierFsCode',axis = 1)
    df = df.drop('primaryCarrierFsCode',axis = 1)
    df = df.drop('carrierFsCode',axis = 1)
    df = df.drop('flightId',axis = 1)
    df = df.drop('codeshares',axis = 1)
    df = df.drop('operationalTimes',axis = 1)
    df = df.drop('schedule',axis = 1)
    df = df.drop('irregularOperations',axis = 1)
    df = df.drop('divertedAirportFsCode', axis = 1)
    df['aircraft_model'] = df['flightEquipment'].apply(lambda x: get_flightModel(x))
    df['tailnumber'] = df['flightEquipment'].apply(lambda x: get_tailnumber(x))
    df = df.drop('flightEquipment',axis=1)
    df['flights_time'] = df['flightDurations'].apply(lambda x: get_flight_time(x))
    df = df.drop('flightDurations',axis = 1)
    df['departureTerminal'] = df['airportResources'].apply(lambda x: get_departure_terminal(x))
    df['arrivalTerminal'] = df['airportResources'].apply(lambda x: get_arival_terminal(x))
    df = df.drop('airportResources',axis=1)
    df = get_utc_from_dict(df,'departureDate')
    df = get_utc_from_dict(df,'arrivalDate')
    #df = pd.get_dummies(df,'aircraft_model')

    return df

In [706]:
flights_df = get_dataframe_after_prossess(flights_df)

## Add more features


Add "is_crestian_holiday"

In [707]:
def add_is_crestian_holiday_column(df):
    df['is_crestian_holiday'] = df['departureDate'].apply(is_christian_holiday)
    return df

In [708]:
def add_advanced_features(df):
    df = add_is_crestian_holiday_column(df)
    df = proccess_date(df,'departureDate')
    df = proccess_date(df,'arrivalDate')
    return df

In [709]:
flights_df = add_advanced_features(flights_df)

C:\Users\rongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  


In [710]:
flights_df.head()

,flightNumber,departureAirportFsCode,arrivalAirportFsCode,delay_time,aircraft_model,tailnumber,flights_time,departureTerminal,arrivalTerminal,is_crestian_holiday,departureDate_year,departureDate_month,departureDate_day,arrivalDate_year,arrivalDate_month,arrivalDate_day
0,1789,PAE,TLV,0,789,4X-EDA,748.0,NaN,3,0,2017,8,22,2017,8,23
1,215,TLV,LHR,0,789,4X-EDA,289.0,3,4,0,2017,9,4,2017,9,4
2,216,LHR,TLV,0,789,4X-EDA,248.0,4,3,0,2017,9,4,2017,9,4
3,225,TLV,CDG,0,789,4X-EDA,NaN,3,2A,0,2017,9,6,2017,9,6
4,226,CDG,TLV,0,789,4X-EDA,NaN,2A,3,0,2017,9,6,2017,9,7


In [711]:
def drop_da_rows(df : pd.DataFrame):
    df = df.dropna(subset=['departureTerminal','arrivalTerminal'])
    return df

## Drop rows with None on required columns

In [712]:
flights_df = drop_da_rows(flights_df)

In [713]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11846 entries, 1 to 15814
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   flightNumber            11846 non-null  object 
 1   departureAirportFsCode  11846 non-null  object 
 2   arrivalAirportFsCode    11846 non-null  object 
 3   delay_time              11846 non-null  int64  
 4   aircraft_model          11846 non-null  object 
 5   tailnumber              11846 non-null  object 
 6   flights_time            11657 non-null  float64
 7   departureTerminal       11846 non-null  object 
 8   arrivalTerminal         11846 non-null  object 
 9   is_crestian_holiday     11846 non-null  int64  
 10  departureDate_year      11846 non-null  int64  
 11  departureDate_month     11846 non-null  int64  
 12  departureDate_day       11846 non-null  int64  
 13  arrivalDate_year        11846 non-null  int64  
 14  arrivalDate_month       11846 non-null

## Convert Text to numbers

In [714]:
def convert_text_to_number(df):
    df['departureTerminal'] = df['departureAirportFsCode'] + '_'+ df['departureTerminal']
    df =  pd.get_dummies( df,columns=['departureAirportFsCode','arrivalAirportFsCode','tailnumber','aircraft_model'])
    return df 

In [715]:
flights_df = convert_text_to_number(flights_df)
flights_df

,flightNumber,delay_time,flights_time,departureTerminal,arrivalTerminal,is_crestian_holiday,departureDate_year,departureDate_month,departureDate_day,arrivalDate_year,...,aircraft_model_73H,aircraft_model_73J,aircraft_model_744,aircraft_model_747,aircraft_model_763,aircraft_model_772,aircraft_model_788,aircraft_model_789,aircraft_model_B738,aircraft_model_B744
1,215,0,289.0,TLV_3,4,0,2017,9,4,2017,...,0,0,0,0,0,0,0,1,0,0
2,216,0,248.0,LHR_4,3,0,2017,9,4,2017,...,0,0,0,0,0,0,0,1,0,0
3,225,0,NaN,TLV_3,2A,0,2017,9,6,2017,...,0,0,0,0,0,0,0,1,0,0
4,226,0,NaN,CDG_2A,3,0,2017,9,6,2017,...,0,0,0,0,0,0,0,1,0,0
5,315,0,272.0,TLV_3,4,0,2017,9,12,2017,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15800,552,21,129.0,SOF_2,3,0,2016,7,18,2016,...,0,0,0,0,0,0,0,0,0,0
15805,551,26,126.0,TLV_3,2,0,2016,7,19,2016,...,0,0,0,0,0,0,0,0,0,0
15806,552,6,124.0,SOF_2,3,0,2016,7,19,2016,...,0,0,0,0,0,0,0,0,0,0
15808,564,3,213.0,WAW_A,3,0,2016,7,20,2016,...,0,0,0,0,0,0,0,0,0,0
